In [2]:
import openai
import pandas as pd
import numpy as np
import os
import json
import random
import nltk
from nltk.corpus import stopwords
from google_pygram import GooglePyGram as gpg
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from string import punctuation

In [3]:
stoplist = set(stopwords.words('english'))

In [6]:
MODEL = "gpt-4-1106-preview"

In [7]:
samples = pd.read_json("/Users/yotmakhova/Downloads/coref_sample.jsonl", lines = True)

In [8]:
samples = samples.to_dict('records')

In [27]:
system_instruction = '''You are a linguist specializing in doing text annotation in the English language. You will be tasked with making minimal modification to a given piece of text based on some linguistics aspects to expose biases in machine learning models.

The given text are samples in the coreference resolution task. 

Each sample has three fields: 
- Text: the input text
- Object: the pronoun or object of interest that we want to find out what it refer to.
- Refers to: the text span that object refers to. Span can either be a noun phrase in the input text, or ambiguous, meaning that it can not be concluded what object refers to.

Make sure to keep the literal string value of Object the same when making modification.

A task may ask for one or multiple modifications. Each modification should be an object with 5 fields: 
type: the type of modification
modified_text: the modified text.
object: the pronoun or object of interest that we want to find out what it refer to.
refer_to: the text span that object refers to.
rationale: the reason why and how the modifications are made.

Please return a json object which consists of one or multiple modifications.
'''

In [10]:
def reformat_sample(sample):
    text = sample['text']
    sample['object'] = sample['label'][0][0]
    sample['span'] = sample['label'][0][1]
    
    return sample

In [11]:
for i,sample in enumerate(samples):
    samples[i] = reformat_sample(sample)
    print(samples[i])

{'text': 'The manager asked the employee to not forget his car.', 'label': [['his', 'ambiguous']], 'object': 'his', 'span': 'ambiguous'}
{'text': 'After meeting with the producers, Sam went to their office.', 'label': [['their', 'ambiguous']], 'object': 'their', 'span': 'ambiguous'}
{'text': 'The trophy didnt fit into the suitcase because it was too small.', 'label': [['it', 'the suitcase']], 'object': 'it', 'span': 'the suitcase'}
{'text': 'The assistant asked the physician if she could borrow some money.', 'label': [['she', 'The assistant']], 'object': 'she', 'span': 'The assistant'}
{'text': "Indeed, Buster was fully intended to exist in place of Spike for the comic book series, until the release of the Fortress Maximus toy in 1987, which included Spike as a Headmaster partner, hence necessitating the hurried introduction of Spike into the comic book continuity. Returning home from college to discover that his father's garage had been destroyed, Spike investigated the Autobots' dese

In [12]:
def format_prompt(system_instruction, instruction, input):
    # print(input)
    text = input['text']
    label = str(input['label'])
    object = input['object']
    span = input['span']
    # print(text)
    # print(label)
    message = instruction.format(text = text, object = object, span = span)
    messages = [
        {"role": "system", "content": system_instruction},
        {"role": "user", "content": message},
    ]
    return messages

In [13]:
# Example OpenAI Python library request
def request(samples, prompt_type):
    for sample in samples:
        messages = format_prompt(system_instruction, prompt_type, sample)
        response = openai.ChatCompletion.create(
            model=MODEL,
            response_format={ "type": "json_object" },
            messages= messages,
            temperature=0,
        )
        ans_model = response['choices'][0]['message']['content']
        # print(messages[1]['content'])
        print(sample)
        print(ans_model)
        print('===================')

Alternative requests and helper functions for the special cases which require another variable for randomization (geographical bias and nonce words)


Geographical bias

In [122]:
subsaharan_africa = ['Angola', 'Benin', 'Botswana', 'Burkina Faso', 'Burundi', 'Cameroon', 'Cape Verde',
          'Central African Republic', 'Chad', 'Comoros', 'Djibouti', 'Republic of the Congo', 'Democratic Republic of the Congo (Zaire)',
          "Côte d'Ivoire", 'Equatorial Guinea', 'Eritrea', 'Ethiopia', 'Gabon', 'Gambia', 'Ghana', 'Guinea', 
          'Guinea-Bissau', 'Kenya', 'Lesotho', 'Liberia', 'Madagascar', 'Malawi', 'Mali', 'Mauritius', 'Mozambique', 
          'Namibia', 'Niger', 'Nigeria', 'Rwanda', 'São Tomé and Príncipe', 'Senegal', 'Seychelles', 'Sierra Leone', 
          'Somalia', 'South Africa', 'South Sudan', 'Swaziland', 'Tanzania', 'Togo', 'Uganda']
          

#Middle East, North Africa, and Turkey		   
menat = ['Algeria', 'Bahrain', 'Iran', 'Iraq', 'Jordan', 'Egypt', 'Kuwait', 'Lebanon', 'Libya', 'Mauritania',
          'Morocco', 'Oman', 'Palestine', 'Qatar', 'Saudi Arabia', 'Sudan', 'Syria', 'Turkey',
          'Tunisia', 'United Arab Emirates', 'Yemen']


southeast_asia = ['Brunei', 'Cambodia', 'Timor Leste', 'Indonesia', 'Laos', 'Malaysia', 'Myanmar',
        'Philippines', 'Singapore', 'Thailand', 'Vietnam']

east_asia = ['China', 'Japan', 'Mongolia', 'North Korea', 'South Korea', 'Taiwan']

south_asia = ['Bangladesh', 'Bhutan', 'India', 'The Maldives', 'Nepal', 'Pakistan', 'Sri Lanka']

central_asia = ['Afghanistan', 'Armenia', 'Azerbaijan', 'Georgia', 'Kazakhstan', 'Kyrgyzstan', 
        'Tajikistan', 'Turkmenistan', 'Uzbekistan']


#Oceania, Melanesia, and Polynesia
oceania = ['Fiji', 'Federated States of Micronesia', 'Kiribati', 'Marshall Islands', 'Nauru', 
           'Palau', 'Papua New Guinea', 'Samoa', 'Solomon Islands', 'Tonga', 'Tuvalu', 'Vanuatu']

australia_nz = ['Australia', 'New Zealand']


#Carribean and Latin America
latin_america = ['Mexico', 'Puerto Rico', 'Dominican Republic', 'Cuba', 'Haiti', 'Belize', 'Grenada', 'Saint Lucia',
           'Costa Rica', 'El Salvador', 'Guatemala', 'Honduras', 'Nicaragua', 'Panama', 'Jamaica', 'Bahamas', 'Barbados',
           'Dominica', 'Brazil', 'Argentina', 'Bolivia', 'Chile', 'Colombia', 'Ecuador', 'Guyana', 'Paraguay', 'Peru',
           'Suriname', 'Trinidad and Tobago', 'Uruguay', 'Venezuela', 'Antigua and Barbuda', 'Saint Kitts and Nevis']

north_america = ['Canada', 'United States of America']

#Northern Europe (nordic)
northern_europe = ['Denmark', 'Estonia', 'Finland', 'Iceland', 'Latvia', 'Lithuania', 'Norway', 'Sweden']
		   
western_europe = ['Belgium', 'France', 'Republic of Ireland', 'Luxembourg', 'Monaco', 'Netherlands', 'United Kingdom']

central_europe = ['Austria', 'Czech Republic', 'Germany', 'Hungary', 'Liechtenstein', 'Poland', 'Slovakia', 'Switzerland']

southern_europe = ['Andorra', 'Italy', 'Malta', 'Portugal', 'San Marino', 'Spain', 'Vatican City']

#Southeastern Europe (mostly Balkan)
southeastern_europe = ['Albania', 'Bosnia and Herzegovina', 'Bulgaria', 'Croatia', 'Cyprus', 'Greece',
          'Kosovo', 'North Macedonia', 'Moldova', 'Montenegro', 'Romania', 'Serbia', 'Slovenia']
		   
eastern_europe = ['Russia', 'Belarus', 'Ukraine']
regions = [subsaharan_africa, menat, south_asia, oceania, latin_america, eastern_europe]

In [135]:
def format_prompt_loc(system_instruction, instruction, input, region):
    # print(input)
    text = input['text']
    label = str(input['label'])
    object = input['object']
    span = input['span']
    loc = random.sample(region,1)
    print(loc)
    message = instruction.format(text = text, object = object, span = span, loc=loc)
    messages = [
        {"role": "system", "content": system_instruction},
        {"role": "user", "content": message},
    ]
    return messages

In [134]:
def request_loc(samples, prompt_type):
    for sample in samples:
        for region in regions:
            print(region)
            messages = format_prompt_loc(system_instruction, prompt_type, sample, region)
            response = openai.ChatCompletion.create(
                model=MODEL,
                response_format={ "type": "json_object" },
                messages= messages,
                temperature=0,
            )
            ans_model = response['choices'][0]['message']['content']
            print(sample)
            print(ans_model)
        print('===================')

Nonce words

In [81]:
nonce_words = ["roagly", "vibble", "drok", "scrop", "plard", "hif", "tepable", "plawic", "bluth", "sprat", "flurf"]

In [87]:
def format_prompt_concepts(system_instruction, instruction, input):
    
    text = input['text']
    label = str(input['label'])
    object = input['object']
    span = input['span']
    nonce = random.sample(nonce_words,1)
    message = instruction.format(text = text, nonce = nonce, object = object, span = span)
    messages = [
        {"role": "system", "content": system_instruction},
        {"role": "user", "content": message},
    ]
    return messages

In [89]:
def request_concepts(samples, prompt_type):
    for sample in samples:
        messages = format_prompt_concepts(system_instruction, prompt_type, sample)
        response = openai.ChatCompletion.create(
            model=MODEL,
            response_format={ "type": "json_object" },
            messages= messages,
            temperature=0,
        )
        ans_model = response['choices'][0]['message']['content']
        # print(messages[1]['content'])
        print(sample)
        print(ans_model)
        print('===================')

# Bias tests

### Frequency bias

In [21]:
frequency_bias_instruction = """Replace one word of higher frequency in English vocabulary with a less frequent word. Add "replaced_word" and "new_word" fields to json output with the replaced and new words, accordingly.

Example:

Text: The cat licked its paw.
Modified Text: The yak licked its paw.


Text: {text}
Object: {object}
Refers to: {span}"""

In [22]:
request(samples, frequency_bias_instruction)

{'text': 'The manager asked the employee to not forget his car.', 'label': [['his', 'ambiguous']], 'object': 'his', 'span': 'ambiguous'}
{
  "modifications": [
    {
      "type": "word_replacement",
      "modified_text": "The supervisor asked the employee to not forget his car.",
      "object": "his",
      "refer_to": "ambiguous",
      "rationale": "The word 'manager' is replaced with 'supervisor' to decrease the frequency of the word while maintaining the meaning of the sentence.",
      "replaced_word": "manager",
      "new_word": "supervisor"
    }
  ]
}
{'text': 'After meeting with the producers, Sam went to their office.', 'label': [['their', 'ambiguous']], 'object': 'their', 'span': 'ambiguous'}
{
  "modifications": [
    {
      "type": "word_replacement",
      "modified_text": "After conferring with the producers, Sam went to their office.",
      "object": "their",
      "refer_to": "ambiguous",
      "rationale": "The word 'meeting' is replaced with 'conferring' to use

KeyboardInterrupt: 

### Temporal bias

In [23]:
temporal_bias_instruction = """Please replace one word with its old-fashioned synonym. 

Text: {text}
Object: {object}
Refers to: {span}

""" 


In [24]:
request(samples, temporal_bias_instruction)

{'text': 'The manager asked the employee to not forget his car.', 'label': [['his', 'ambiguous']], 'object': 'his', 'span': 'ambiguous'}
{
  "modifications": [
    {
      "type": "lexical_substitution",
      "modified_text": "The manager asked the employee to not forget his automobile.",
      "object": "his",
      "refer_to": "ambiguous",
      "rationale": "The word 'car' has been replaced with 'automobile', which is an old-fashioned synonym. This modification was made to test if the machine learning model's coreference resolution is sensitive to lexical changes, even when the meaning is preserved. The referent of 'his' remains ambiguous as the text does not clarify whether 'his' refers to the manager or the employee."
    }
  ]
}
{'text': 'After meeting with the producers, Sam went to their office.', 'label': [['their', 'ambiguous']], 'object': 'their', 'span': 'ambiguous'}
{
  "modification_1": {
    "type": "lexical_substitution",
    "modified_text": "After meeting with the pr

KeyboardInterrupt: 

### Geographical bias

In [132]:
geographical_bias_instruction = """Change the entity name in the sentence with name commonly used in {loc}. 
Change the other words in the sentence so that it is culturally fitted with context of {loc}.
Do not modify entities mentioned in {span}. Write the modified sentences in English.

Text: {text}
Object: {object}
Refers to: {span}"""

In [133]:
request_loc(samples, geographical_bias_instruction)

['Angola', 'Benin', 'Botswana', 'Burkina Faso', 'Burundi', 'Cameroon', 'Cape Verde', 'Central African Republic', 'Chad', 'Comoros', 'Djibouti', 'Republic of the Congo', 'Democratic Republic of the Congo (Zaire)', "Côte d'Ivoire", 'Equatorial Guinea', 'Eritrea', 'Ethiopia', 'Gabon', 'Gambia', 'Ghana', 'Guinea', 'Guinea-Bissau', 'Kenya', 'Lesotho', 'Liberia', 'Madagascar', 'Malawi', 'Mali', 'Mauritius', 'Mozambique', 'Namibia', 'Niger', 'Nigeria', 'Rwanda', 'São Tomé and Príncipe', 'Senegal', 'Seychelles', 'Sierra Leone', 'Somalia', 'South Africa', 'South Sudan', 'Swaziland', 'Tanzania', 'Togo', 'Uganda']
{'text': 'The manager asked the employee to not forget his car.', 'label': [['his', 'ambiguous']], 'object': 'his', 'span': 'ambiguous'}
{
  "modification_1": {
    "type": "entity_name_change",
    "modified_text": "The supervisor asked the worker to not forget his moto.",
    "object": "his",
    "refer_to": "ambiguous",
    "rationale": "The name 'manager' was changed to 'supervisor'

{'text': 'After meeting with the producers, Sam went to their office.', 'label': [['their', 'ambiguous']], 'object': 'their', 'span': 'ambiguous'}
{
  "modifications": [
    {
      "type": "entity_name_cultural_adaptation",
      "modified_text": "After meeting with the growers, Nsabimana went to their office.",
      "object": "their",
      "refer_to": "ambiguous",
      "rationale": "The name 'Sam' was changed to 'Nsabimana', which is a name commonly used in Burundi. The term 'producers' was changed to 'growers' to better fit a context that might be more culturally relevant to Burundi, which has a significant agricultural sector. The rest of the sentence was kept the same, including the ambiguous reference of 'their', as the task specified not to modify entities mentioned as ambiguous."
    }
  ]
}
['Algeria', 'Bahrain', 'Iran', 'Iraq', 'Jordan', 'Egypt', 'Kuwait', 'Lebanon', 'Libya', 'Mauritania', 'Morocco', 'Oman', 'Palestine', 'Qatar', 'Saudi Arabia', 'Sudan', 'Syria', 'Turkey',

{'text': 'The trophy didnt fit into the suitcase because it was too small.', 'label': [['it', 'the suitcase']], 'object': 'it', 'span': 'the suitcase'}
{
  "modification_1": {
    "type": "entity_name_change",
    "modified_text": "The award didn't fit into the handwoven basket because it was too small.",
    "object": "it",
    "refer_to": "the handwoven basket",
    "rationale": "The entity 'trophy' has been changed to 'award' to maintain neutrality, and 'suitcase' has been changed to 'handwoven basket' to reflect an object that might be more commonly used in Mauritania. The rest of the sentence has been kept the same to maintain the original meaning."
  },
  "modification_2": {
    "type": "cultural_contextualization",
    "modified_text": "The award didn't fit into the handwoven basket because it was too small.",
    "object": "it",
    "refer_to": "the handwoven basket",
    "rationale": "The sentence has been modified to include 'handwoven basket' instead of 'suitcase' to cultura

KeyboardInterrupt: 

### Position bias

In [30]:
position_bias_instruction = '''Move the text that the pronoun {object} refers to ({span}) to another position in the same sentence.

Text: {text}
Object: {object}
Refers to: {span}

'''

In [31]:
request(samples, position_bias_instruction)

{'text': 'The manager asked the employee to not forget his car.', 'label': [['his', 'ambiguous']], 'object': 'his', 'span': 'ambiguous'}
{
  "modifications": [
    {
      "type": "repositioning",
      "modified_text": "The manager asked the employee to not forget the car of his.",
      "object": "his",
      "refer_to": "ambiguous",
      "rationale": "The possessive pronoun 'his' has been moved to the end of the sentence in the form of 'the car of his' to maintain ambiguity, as it is still unclear whether 'his' refers to the manager or the employee."
    }
  ]
}
{'text': 'After meeting with the producers, Sam went to their office.', 'label': [['their', 'ambiguous']], 'object': 'their', 'span': 'ambiguous'}
{
  "modifications": [
    {
      "type": "repositioning",
      "modified_text": "After meeting with the producers in their office, Sam went there.",
      "object": "their",
      "refer_to": "the producers",
      "rationale": "The modification moves the possessive 'their' to

KeyboardInterrupt: 

### Length Bias 

In [34]:
length_bias_instruction = '''Modify the sentence length, but retain sentence meaning. Do not remove or modify the following text: {span}.

1. "shorter_sentence": Remove 1 - 3 words from the sentence.
2. "longer_sentence": Add 2 - 5 words to the sentence.

Text: {text}
Object: {object}
Refers to: {span}

'''

In [35]:
request(samples, length_bias_instruction)

{'text': 'The manager asked the employee to not forget his car.', 'label': [['his', 'ambiguous']], 'object': 'his', 'span': 'ambiguous'}
{
  "modifications": [
    {
      "type": "shorter_sentence",
      "modified_text": "The manager asked the employee to remember his car.",
      "object": "his",
      "refer_to": "ambiguous",
      "rationale": "The phrase 'to not forget' was replaced with 'to remember' to reduce the word count by two while retaining the original meaning of the sentence."
    },
    {
      "type": "longer_sentence",
      "modified_text": "The manager asked the employee to not forget his car keys today.",
      "object": "his",
      "refer_to": "ambiguous",
      "rationale": "The words 'keys today' were added to the sentence to increase the word count by three and add specificity while maintaining the original sentence meaning."
    }
  ]
}
{'text': 'After meeting with the producers, Sam went to their office.', 'label': [['their', 'ambiguous']], 'object': 'their

KeyboardInterrupt: 

## Orthography

### Typos

In [36]:
typo_instruction = '''Add typos to the text. Common types of typos are:

1. "addition": Adding a letter: Forty (correct) vs. Fourty
2. "omission": Omitting a letter: Embarrass (correct) vs. Embarass
3. "flipping": flipping letters: Friend (correct) vs. Freind

Reply with the modified text for each type of typo. Do not modify the following words: {object}, {span}

Text: {text}
Object: {object}
Refers to: {span}

'''

In [37]:
request(samples, typo_instruction)

{'text': 'The manager asked the employee to not forget his car.', 'label': [['his', 'ambiguous']], 'object': 'his', 'span': 'ambiguous'}
{
  "modifications": [
    {
      "type": "addition",
      "modified_text": "The managger asked the employee to not forget his car.",
      "object": "his",
      "refer_to": "ambiguous",
      "rationale": "Added an extra 'g' in the word 'manager' to create a common addition typo."
    },
    {
      "type": "omission",
      "modified_text": "The manger asked the employee to not forget his car.",
      "object": "his",
      "refer_to": "ambiguous",
      "rationale": "Omitted the 'a' in the word 'manager' to create a common omission typo."
    },
    {
      "type": "flipping",
      "modified_text": "The manager asked the employe to not forget his car.",
      "object": "his",
      "refer_to": "ambiguous",
      "rationale": "Flipped the 'e' and the 'e' in the word 'employee' to create a common flipping typo, resulting in the word 'employe'."
 

KeyboardInterrupt: 

## Capitalization

In [40]:
capitalization_instruction = '''Modify the text capitalization by:

1. "lower": change a word with upper case to lower case.

2. "upper": change a word starting with lower case to upper case.

3. "all_caps": change a word to ALL CAPS.

4. "sponge": change a word to SpOnGeBoBcASe.

Reply with the modified text for each type of change.

Text: {text}
Object: {object}
Refers to: {span}

'''

In [41]:
request(samples, capitalization_instruction)

{'text': 'The manager asked the employee to not forget his car.', 'label': [['his', 'ambiguous']], 'object': 'his', 'span': 'ambiguous'}
{
  "modifications": [
    {
      "type": "lower",
      "modified_text": "the manager asked the employee to not forget his car.",
      "object": "his",
      "refer_to": "ambiguous",
      "rationale": "The first letter of the sentence 'The' is changed to lowercase to demonstrate the effect of lowercasing a word that typically starts with an uppercase letter."
    },
    {
      "type": "upper",
      "modified_text": "The Manager asked the employee to not forget his car.",
      "object": "his",
      "refer_to": "ambiguous",
      "rationale": "The word 'manager' is changed to 'Manager' with an uppercase 'M' to demonstrate the effect of capitalizing a word that typically starts with a lowercase letter."
    },
    {
      "type": "all_caps",
      "modified_text": "The MANAGER asked the employee to not forget his car.",
      "object": "his",
   

KeyboardInterrupt: 

## Punctuation

In [44]:
punctuation_instruction = '''Make change to the punctuation of the text:

1. "addition": add a random comma, semi-colon etc 

2. "replacement": change the full stop to the exclamation or question mark 

3. "glueing": remove white space between two words (glue them together). If possible, glue the word which the pronoun {object} refers to.

Reply with the modified text for each type of change.

Text: {text}
Object: {object}
Refers to: {span}

'''

In [45]:
request(samples, punctuation_instruction)

{'text': 'The manager asked the employee to not forget his car.', 'label': [['his', 'ambiguous']], 'object': 'his', 'span': 'ambiguous'}
{
  "modifications": [
    {
      "type": "addition",
      "modified_text": "The manager, asked the employee to not forget his car.",
      "object": "his",
      "refer_to": "ambiguous",
      "rationale": "A comma was added after 'manager' to introduce a pause, which is not grammatically necessary but changes the flow of the sentence."
    },
    {
      "type": "replacement",
      "modified_text": "The manager asked the employee to not forget his car!",
      "object": "his",
      "refer_to": "ambiguous",
      "rationale": "The full stop at the end of the sentence was replaced with an exclamation mark to change the tone of the sentence to convey more urgency or emotion."
    },
    {
      "type": "glueing",
      "modified_text": "The manager asked the employee to not forget hiscar.",
      "object": "his",
      "refer_to": "ambiguous",
    

KeyboardInterrupt: 

## Morphology

## Derivation

In [56]:
derivation_replacement_instruction ='''Find a derived word (a word with a suffix or a prefix) in the text below and replace it with a non-derived word (word without any prefixes or suffixes). Do not modify the following words: {span}.


Example: a sometimes tedious film -> a sometimes dull film (tedious is derived from tedium using a -ios suffix)

Example: a very good film -> Skip

Text: {text}
Object: {object}
Refers to: {span}'''

In [57]:
request(samples, derivation_replacement_instruction)

{'text': 'The manager asked the employee to not forget his car.', 'label': [['his', 'ambiguous']], 'object': 'his', 'span': 'ambiguous'}
{
  "type": "lexical_substitution",
  "modified_text": "The manager asked the worker to not forget his car.",
  "object": "his",
  "refer_to": "ambiguous",
  "rationale": "The word 'employee' is derived from the verb 'employ' with the suffix '-ee' indicating a person who is the object of an action. It has been replaced with the non-derived word 'worker'."
}
{'text': 'After meeting with the producers, Sam went to their office.', 'label': [['their', 'ambiguous']], 'object': 'their', 'span': 'ambiguous'}
{
  "type": "lexical_substitution",
  "modified_text": "After meeting with the producers, Sam went to the studio.",
  "object": "their",
  "refer_to": "ambiguous",
  "rationale": "The word 'producers' is a derived word, with the suffix '-ers' indicating a group of people who produce. To remove the derivation, the word 'producers' is replaced with 'studio

KeyboardInterrupt: 

In [60]:
antiderivation_replacement_instruction ='''Find any non-derived word (a word without suffixes or prefixes) in the text below and change it into a derived word (word with a prefix or a suffix). Do not add grammatical suffixes (-s, -ed, -er, -est). Do not modify the following words: {span}.

Example: a sometimes dull film ->  a sometimes tedious film (tedious is derived from tedium using a -ios suffix)

Example: an very hard task -> an increasingly hard task (increasingly is derived from increasing using a -ly suffix)

Example: amazing accomplishment -> Skip (both words are already derived)

Text: {text}
Object: {object}
Refers to: {span}'''

In [61]:
request(samples, antiderivation_replacement_instruction)

{'text': 'The manager asked the employee to not forget his car.', 'label': [['his', 'ambiguous']], 'object': 'his', 'span': 'ambiguous'}
{
  "modifications": [
    {
      "type": "lexical_derivation",
      "modified_text": "The managerial asked the employee to not forget his car.",
      "object": "his",
      "refer_to": "ambiguous",
      "rationale": "The non-derived word 'manager' was changed to 'managerial', which is a derived form using the -ial suffix."
    },
    {
      "type": "lexical_derivation",
      "modified_text": "The manager asked the employee to not disregard his car.",
      "object": "his",
      "refer_to": "ambiguous",
      "rationale": "The non-derived word 'forget' was changed to 'disregard', which is a derived form using the dis- prefix."
    }
  ]
}
{'text': 'After meeting with the producers, Sam went to their office.', 'label': [['their', 'ambiguous']], 'object': 'their', 'span': 'ambiguous'}
{
  "modifications": [
    {
      "type": "lexical_derivation

KeyboardInterrupt: 

## Compound words

In [62]:
compound_word_instruction ='''Find any non-compond (single-root) word in the text below and change it into a compound word (word with several roots). Do not modify the following words: {span}.


Example: 
"a sequence of ridiculous shooting scenes" -> "a sequence of ridiculous shoot-'em-up scenes"
Example: dull acting ->  lacklustre acting

Text: {text}
Object: {object}
Refers to: {span}
'''

In [63]:
request(samples, compound_word_instruction)

{'text': 'The manager asked the employee to not forget his car.', 'label': [['his', 'ambiguous']], 'object': 'his', 'span': 'ambiguous'}
{
  "modifications": [
    {
      "type": "compound_word_insertion",
      "modified_text": "The manager asked the employee to not forget his sports car.",
      "object": "his",
      "refer_to": "ambiguous",
      "rationale": "The word 'car' was changed to 'sports car' to introduce a compound word. The modification does not clarify the antecedent of 'his', thus maintaining the ambiguity."
    }
  ]
}
{'text': 'After meeting with the producers, Sam went to their office.', 'label': [['their', 'ambiguous']], 'object': 'their', 'span': 'ambiguous'}
{
  "modification_1": {
    "type": "compound_creation",
    "modified_text": "After meeting with the co-producers, Sam went to their office.",
    "object": "their",
    "refer_to": "ambiguous",
    "rationale": "The word 'producers' was modified to 'co-producers' to create a compound word while maintainin

KeyboardInterrupt: 

## Irregular verbs

In [64]:
regular_verb_instruction ='''Find a regular verb in the past tense and replace it with an irregular verb.

Example: he received a prize -> he got a prize

Example: amazing stuff  ->  Skip

Text: {text}
Object: {object}
Refers to: {span}'''

In [65]:
request(samples, regular_verb_instruction)

{'text': 'The manager asked the employee to not forget his car.', 'label': [['his', 'ambiguous']], 'object': 'his', 'span': 'ambiguous'}
{
  "modification": [
    {
      "type": "verb_change",
      "modified_text": "The manager told the employee to not forget his car.",
      "object": "his",
      "refer_to": "ambiguous",
      "rationale": "The regular verb 'asked' was replaced with the irregular verb 'told' to maintain past tense while changing the verb form."
    }
  ]
}
{'text': 'After meeting with the producers, Sam went to their office.', 'label': [['their', 'ambiguous']], 'object': 'their', 'span': 'ambiguous'}
{
  "modifications": [
    {
      "type": "verb_change",
      "modified_text": "After meeting with the producers, Sam drove to their office.",
      "object": "their",
      "refer_to": "ambiguous",
      "rationale": "The regular verb 'went' was replaced with the irregular verb 'drove' to maintain past tense while changing the verb form."
    }
  ]
}
{'text': 'The t

KeyboardInterrupt: 

## Syntax

### Active to passive

In [66]:
passive_voice_instruction = '''Rewrite the sentence in passive voice.

Text: {text}
Object: {object}
Refers to: {span}

'''

In [67]:
request(samples, passive_voice_instruction)

{'text': 'The manager asked the employee to not forget his car.', 'label': [['his', 'ambiguous']], 'object': 'his', 'span': 'ambiguous'}
{
  "type": "passive_voice_transformation",
  "modified_text": "The employee was asked by the manager to not forget his car.",
  "object": "his",
  "refer_to": "ambiguous",
  "rationale": "The sentence has been rewritten in passive voice by making the object ('the employee') the subject of the sentence, and the subject ('the manager') follows the verb phrase with 'by'. The verb 'asked' is used with 'was' to form the passive voice. The pronoun 'his' remains ambiguous as it still could refer to either the manager or the employee."
}
{'text': 'After meeting with the producers, Sam went to their office.', 'label': [['their', 'ambiguous']], 'object': 'their', 'span': 'ambiguous'}
{
  "type": "passive_voice",
  "modified_text": "After being met by the producers, Sam was gone to their office.",
  "object": "their",
  "refer_to": "ambiguous",
  "rationale": "

KeyboardInterrupt: 

### Coordinating conjunctions

In [68]:
coordinating_conjunction_instruction = '''Change a noun or verb in this sentence into multiple nouns or verbs combined with coordinating conjunction. Do not modify the following words: {span}

Text: {text}
Object: {object}
Refers to: {span}'''

In [69]:
request(samples, coordinating_conjunction_instruction)

{'text': 'The manager asked the employee to not forget his car.', 'label': [['his', 'ambiguous']], 'object': 'his', 'span': 'ambiguous'}
{
  "modification": {
    "type": "coordinating_conjunction",
    "modified_text": "The manager asked the employee to not forget his car or his documents.",
    "object": "his",
    "refer_to": "ambiguous",
    "rationale": "The noun 'car' was expanded into a compound object 'car or documents' using the coordinating conjunction 'or'. This maintains the ambiguity of the pronoun 'his' as it could now refer to either 'his car' or 'his documents', or both, thus preserving the original ambiguity while fulfilling the task requirement."
  }
}
{'text': 'After meeting with the producers, Sam went to their office.', 'label': [['their', 'ambiguous']], 'object': 'their', 'span': 'ambiguous'}
{
  "modification_1": {
    "type": "noun_phrase_modification",
    "modified_text": "After meeting with the producers, Sam went to the office of the director and the manager

KeyboardInterrupt: 

### Tense

In [70]:
tense_instruction = '''Change the tense of verbs in the sentence. Keep the tenses consistent across the passage.


Text: {text}
Object: {object}
Refers to: {span}'''

In [71]:
request(samples, tense_instruction)

{'text': 'The manager asked the employee to not forget his car.', 'label': [['his', 'ambiguous']], 'object': 'his', 'span': 'ambiguous'}
{
  "modifications": [
    {
      "type": "Tense Change",
      "modified_text": "The manager will ask the employee to not forget his car.",
      "object": "his",
      "refer_to": "ambiguous",
      "rationale": "The tense of the verb 'asked' has been changed to 'will ask' to shift the sentence from past to future tense. The verb 'forget' remains in the base form as it is part of an infinitive structure 'to not forget' which does not change with tense. The tense change is applied consistently across the passage."
    }
  ]
}
{'text': 'After meeting with the producers, Sam went to their office.', 'label': [['their', 'ambiguous']], 'object': 'their', 'span': 'ambiguous'}
{
  "modifications": [
    {
      "type": "tense_change",
      "modified_text": "After meeting with the producers, Sam goes to their office.",
      "object": "their",
      "refer

KeyboardInterrupt: 

### Grammatical role

In [72]:
grammatical_role_instruction = '''Modify the position of grammatical role in the sentence.

1. "subject_object": swap the subject with object
Example: 
Miranda asked him a question. --> He asked Miranda a question.

2. "entities": swap the position of other entities
Example:
Samantha, the older Rico's friend, will be appointed as Chair of Student Body replacing Marie. --> Marie, the older Samantha's friend, will be appointed as Chair of Student Body replacing Rico.

Text: {text}
Object: {object}
Refers to: {span}'''

In [73]:
request(samples, grammatical_role_instruction)

{'text': 'The manager asked the employee to not forget his car.', 'label': [['his', 'ambiguous']], 'object': 'his', 'span': 'ambiguous'}
{
  "modifications": [
    {
      "type": "subject_object",
      "modified_text": "The employee asked the manager to not forget his car.",
      "object": "his",
      "refer_to": "ambiguous",
      "rationale": "The subject 'The manager' and the object 'the employee' have been swapped to change the grammatical roles in the sentence. The referent of 'his' remains ambiguous as it could still refer to either 'the employee' or 'the manager'."
    },
    {
      "type": "entities",
      "modified_text": "The employee asked the car to not forget his manager.",
      "object": "his",
      "refer_to": "ambiguous",
      "rationale": "The position of the entities 'the manager' and 'his car' have been swapped. The sentence now suggests that the employee is speaking to the car, which is nonsensical, but serves to illustrate the change in entity positions. T

KeyboardInterrupt: 

### Clause structure

In [74]:
sentence_structure_instruction = '''
Change the position of main and subordinate clause. If the input is simple sentence, skip.

Text: {text}
Object: {object}
Refers to: {span}'''

In [75]:
request(samples, sentence_structure_instruction)

{'text': 'The manager asked the employee to not forget his car.', 'label': [['his', 'ambiguous']], 'object': 'his', 'span': 'ambiguous'}
{
  "modifications": [
    {
      "type": "clause_position_change",
      "modified_text": "To not forget his car, the manager asked the employee.",
      "object": "his",
      "refer_to": "the manager",
      "rationale": "By changing the position of the main and subordinate clause, the proximity of 'his' to 'the manager' increases, which may influence coreference resolution models to associate 'his' with 'the manager' instead of remaining ambiguous."
    }
  ]
}
{'text': 'After meeting with the producers, Sam went to their office.', 'label': [['their', 'ambiguous']], 'object': 'their', 'span': 'ambiguous'}
{
  "type": "clause_position_change",
  "modified_text": "Sam went to their office after meeting with the producers.",
  "object": "their",
  "refer_to": "ambiguous",
  "rationale": "The position of the main clause ('Sam went to their office') a

KeyboardInterrupt: 

## Semantics and lexicon

## Concept replacement

In [84]:
concept_replacement_instruction ='''Replace a concept with one that is similar but fail the quality. Do not modify the following words: {span}

Replacement types include:

1. "synonym": synonyms
Example: embodies the character with an effortlessly regal charisma . -> embodies the character with an effortlessly regal charm. (charm and charisma are synonyms)

2. "hierarchy": hyper/hyponyms
Example: The title not only describes its main characters, but the lazy people behind the camera as well. -> The title not only describes its main characters, but the lazy people behind the equipment as well. (equipment is a hypernym of camera)

3. "nonce": the following nonce word: {nonce}
Example: Has a lot of the virtues of eastwood at his best. - Has a lot of the virtues of bluth at his best. (bluth is a nonce word)

4. "idiom": metaphors/idioms
Example: This is a train wreck of an action film -> This is a disastrous action film (train wreck is a metaphor for disastrous)

Reply the modified text for each type of replacement.

Text: {text}
Object: {object}
Refers to: {span}

'''

In [90]:
request_concepts(samples, concept_replacement_instruction)

{'text': 'The manager asked the employee to not forget his car.', 'label': [['his', 'ambiguous']], 'object': 'his', 'span': 'ambiguous'}
{
  "modifications": [
    {
      "type": "synonym",
      "modified_text": "The manager asked the worker to not forget his car.",
      "object": "his",
      "refer_to": "ambiguous",
      "rationale": "The word 'employee' was replaced with 'worker', which is a synonym, to maintain the ambiguity of the pronoun 'his'."
    },
    {
      "type": "hierarchy",
      "modified_text": "The manager asked the subordinate to not forget his car.",
      "object": "his",
      "refer_to": "ambiguous",
      "rationale": "The word 'employee' was replaced with 'subordinate', which is a hyponym, to maintain the ambiguity of the pronoun 'his'."
    },
    {
      "type": "nonce",
      "modified_text": "The manager asked the sprat to not forget his car.",
      "object": "his",
      "refer_to": "ambiguous",
      "rationale": "The word 'employee' was replaced w

KeyboardInterrupt: 

## Adjectives/adverbs change

In [95]:
adjective_adverb_remove_instruction = '''Remove an adjective or adverb from the sentence. Do not remove the following words: {object}, {span}

Text: {text}
Object: {object}
Refers to: {span}
'''

In [96]:
request(samples, adjective_adverb_remove_instruction)

{'text': 'The manager asked the employee to not forget his car.', 'label': [['his', 'ambiguous']], 'object': 'his', 'span': 'ambiguous'}
{
  "modifications": [
    {
      "type": "Remove Adjective/Adverb",
      "modified_text": "The manager asked the employee to not forget the car.",
      "object": "his",
      "refer_to": "ambiguous",
      "rationale": "The adverb 'not' was removed to maintain the sentence structure while altering the meaning slightly. The possessive pronoun 'his' remains unchanged, and the reference remains ambiguous as it is unclear if 'the car' refers to the manager's or the employee's car."
    }
  ]
}
{'text': 'After meeting with the producers, Sam went to their office.', 'label': [['their', 'ambiguous']], 'object': 'their', 'span': 'ambiguous'}
{
  "modification_1": {
    "type": "Removal of adjective/adverb",
    "modified_text": "After meeting with the producers, Sam went to the office.",
    "object": "the",
    "refer_to": "ambiguous",
    "rationale": "

KeyboardInterrupt: 

In [93]:
adjective_adverb_add_instruction = '''Add an adjective or an adverb to the sentence.

Text: {text}
Object: {object}
Refers to: {span}
'''

In [94]:
request(samples, adjective_adverb_add_instruction)

{'text': 'The manager asked the employee to not forget his car.', 'label': [['his', 'ambiguous']], 'object': 'his', 'span': 'ambiguous'}
{
  "modifications": [
    {
      "type": "Addition of an adjective",
      "modified_text": "The manager asked the diligent employee to not forget his car.",
      "object": "his",
      "refer_to": "the diligent employee",
      "rationale": "By adding the adjective 'diligent' before 'employee', we clarify that 'his' refers to 'the diligent employee'. This reduces the ambiguity as to whom the car belongs."
    }
  ]
}
{'text': 'After meeting with the producers, Sam went to their office.', 'label': [['their', 'ambiguous']], 'object': 'their', 'span': 'ambiguous'}
{
  "modifications": [
    {
      "type": "Addition of an adjective",
      "modified_text": "After meeting with the talented producers, Sam went to their office.",
      "object": "their",
      "refer_to": "the talented producers",
      "rationale": "By adding the adjective 'talented' b

KeyboardInterrupt: 

## Pragmatics and discourse

### Negation

In [97]:
negation_instruction = '''Negate the text by making minimal modifications to introduce different types of negation. The types of negation include:

1. "verbal": verbal negation: when the negation is grammatically associated with the verb, the head of the clause.
Examples:
He trusts them. => He does not trust them.

2. "absolute": Absolute negator: no (including compounds nobody, nothing, etc., and the independent form none), neither, nor, never.
Example:
He trusts them. => He trusts noone.

3. "approximate": Approximate negators: few, little; barely, hardly, scarcely; rarely, seldom.
Example:
He trusts people. => He rarely trusts people.

4. "affixal": Affixal negators: un-, in-, non-, -less, etc. Do not change the root of the word you add the affix to.
Example:
He is healthy => He is unhealthy.

5. "lexical": Lexical negation: when the negation is added by substituting the main predicate of the sentence with its antonym or word carrying negative meaning.
Examples:
The house is big. => The house is small.

6. "unimportant": Unimportant negation: when the negation does not affect the main clause of the text and does not affect the coreference chain.
Examples:
A man is driving his car. => A shirtless man is driving his car.

7. "double": Double negation: when there are two instances of negation that cancel each other and the meaning is affirmative.

Reply with the modified text for each type of negation.

Text: {text}
Object: {object}
Refers to: {span}
'''

In [98]:
request(samples, negation_instruction)

{'text': 'The manager asked the employee to not forget his car.', 'label': [['his', 'ambiguous']], 'object': 'his', 'span': 'ambiguous'}
{
  "modifications": [
    {
      "type": "verbal",
      "modified_text": "The manager asked the employee to forget his car.",
      "object": "his",
      "refer_to": "ambiguous",
      "rationale": "The verbal negation 'not' is removed to negate the verb 'forget'."
    },
    {
      "type": "absolute",
      "modified_text": "The manager asked the employee to not forget his car for no reason.",
      "object": "his",
      "refer_to": "ambiguous",
      "rationale": "The absolute negator 'no reason' is added to introduce negation without changing the original negated verb."
    },
    {
      "type": "approximate",
      "modified_text": "The manager asked the employee to seldom forget his car.",
      "object": "his",
      "refer_to": "ambiguous",
      "rationale": "The approximate negator 'seldom' is added to introduce negation related to the

KeyboardInterrupt: 

In [100]:
antinegation_instruction = '''Check if the text contains negation, and if it does, remove it. Do not make any other modifications. If there is no negation, output Skip. 


Examples:  

Text: The house is not pretty. 

Modified text: The house is pretty.  

 

Text: The story didn't leave anyone unaffected. 

Modified text: The story left everyone unaffected. 


Text: The house is pretty. 

Modified text: Skip.  


Text: {text}
Object: {object}
Refers to: {span}''' 

In [101]:
request(samples, antinegation_instruction)

{'text': 'The manager asked the employee to not forget his car.', 'label': [['his', 'ambiguous']], 'object': 'his', 'span': 'ambiguous'}
{
  "type": "negation_removal",
  "modified_text": "The manager asked the employee to forget his car.",
  "object": "his",
  "refer_to": "ambiguous",
  "rationale": "The negation 'not' was removed to change the sentence from a negative imperative to a positive one, without altering the meaning of the pronoun 'his'."
}
{'text': 'After meeting with the producers, Sam went to their office.', 'label': [['their', 'ambiguous']], 'object': 'their', 'span': 'ambiguous'}
{
  "type": "negation_removal",
  "modified_text": "The house is pretty.",
  "object": "their",
  "refer_to": "ambiguous",
  "rationale": "The negation 'not' was removed to change the sentence from negative to positive without altering the meaning of the rest of the text."
}
{'text': 'The trophy didnt fit into the suitcase because it was too small.', 'label': [['it', 'the suitcase']], 'object'

KeyboardInterrupt: 

## Discourse markers

In [102]:
discourse_instruction = '''Make modification to the discourse makers in the text. The types of modification include:

1. "addition": Add discourse markers to the sentence.
Example:
He was hungry, he went out to eat => He was hungry, so he went out to eat.

2. "replacement": Change the discourse marker into a different one. If there is no discourse marker, output Skip.
Example:
He was hungry although he had dinner => He was hungry so he had dinner. (although => so)

3. "deletion": Delete the discourse marker. If there is no discourse marker, output Skip.
Example:
He was hungry so he had dinner => He was hungry, he had dinner.

For each type of modification, reply with the modified sentences.

Text: {text}
Object: {object}
Refers to: {span}

'''

In [103]:
request(samples, discourse_instruction)

{'text': 'The manager asked the employee to not forget his car.', 'label': [['his', 'ambiguous']], 'object': 'his', 'span': 'ambiguous'}
{
  "modifications": [
    {
      "type": "addition",
      "modified_text": "The manager asked the employee to not forget his car, because it was important.",
      "object": "his",
      "refer_to": "ambiguous",
      "rationale": "Added the discourse marker 'because' to provide a reason for the manager's request, which is that it was important."
    },
    {
      "type": "replacement",
      "modified_text": "Skip",
      "object": "his",
      "refer_to": "ambiguous",
      "rationale": "There is no discourse marker in the original sentence to replace."
    },
    {
      "type": "deletion",
      "modified_text": "Skip",
      "object": "his",
      "refer_to": "ambiguous",
      "rationale": "There is no discourse marker in the original sentence to delete."
    }
  ]
}
{'text': 'After meeting with the producers, Sam went to their office.', 'la

KeyboardInterrupt: 

## Sentiment

In [104]:
sentiment_instruction = '''Add a word or a phrase with a positive or negative sentiment to the sentence. 

Example: We beat the competition -> We are happy to beat the competition. 

Text: {text}
Object: {object}
Refers to: {span}

'''

In [105]:
request(samples, sentiment_instruction)

{'text': 'The manager asked the employee to not forget his car.', 'label': [['his', 'ambiguous']], 'object': 'his', 'span': 'ambiguous'}
{
  "modification_1": {
    "type": "Addition of positive sentiment",
    "modified_text": "The manager kindly reminded the employee to not forget his car.",
    "object": "his",
    "refer_to": "ambiguous",
    "rationale": "The word 'kindly' adds a positive sentiment to the sentence, suggesting that the manager's request was made in a friendly or considerate manner."
  },
  "modification_2": {
    "type": "Addition of negative sentiment",
    "modified_text": "The manager sternly warned the employee to not forget his car.",
    "object": "his",
    "refer_to": "ambiguous",
    "rationale": "The word 'sternly' adds a negative sentiment to the sentence, implying that the manager's request was made in a serious or strict manner."
  }
}
{'text': 'After meeting with the producers, Sam went to their office.', 'label': [['their', 'ambiguous']], 'object': '

KeyboardInterrupt: 

## Style

### Casual vs formal style

In [106]:
casual_instruction = '''Rewrite the sentence in informal way. Do not modify the following words: {span}

Text: {text}
Object: {object}
Refers to: {span}
'''

In [107]:
request(samples, casual_instruction)

{'text': 'The manager asked the employee to not forget his car.', 'label': [['his', 'ambiguous']], 'object': 'his', 'span': 'ambiguous'}
{
  "modifications": [
    {
      "type": "informal_rewrite",
      "modified_text": "The boss told the worker to remember his car.",
      "object": "his",
      "refer_to": "ambiguous",
      "rationale": "The text has been modified to a more informal tone by changing 'manager' to 'boss' and 'employee' to 'worker'. The phrase 'asked the employee to not forget' has been simplified to 'told the worker to remember' to match informal speech patterns. The literal string value of 'his' remains unchanged, and the reference remains ambiguous."
    }
  ]
}
{'text': 'After meeting with the producers, Sam went to their office.', 'label': [['their', 'ambiguous']], 'object': 'their', 'span': 'ambiguous'}
{
  "modifications": [
    {
      "type": "informalization",
      "modified_text": "After hanging out with the producers, Sam headed to their office.",
     

{'text': "At the Loebner competition the judges couldn't figure out which respondents were the chatbots because they were so stupid.", 'label': [['they', 'the judges']], 'object': 'they', 'span': 'the judges'}
{
  "modifications": [
    {
      "type": "informality",
      "modified_text": "At the Loebner contest, the judges were totally clueless about which ones were the chatbots - they were so out of it.",
      "object": "they",
      "refer_to": "the judges",
      "rationale": "The text has been modified to sound more informal by using colloquial expressions such as 'totally clueless' and 'out of it' to describe the judges' inability to distinguish chatbots. The formal term 'competition' has been replaced with the more casual 'contest'. The structure of the sentence has been slightly altered to maintain a conversational tone while keeping the meaning intact and ensuring that 'they' still refers to 'the judges'."
    }
  ]
}
{'text': "The sculpture rolled off the shelf because it w

KeyboardInterrupt: 

### Simple vs Complex English

### Plain English

In [110]:
plain_english_instruction = '''Please rewrite the sentence in order to make it easier to understand by non-native speakers of English. 
You can do so by replacing complex words with simpler synonyms (i.e. paraphrasing), deleting unimportant information (i.e. compres-sion), and/or splitting a long complex sentence into several simpler ones. The final simplified sentence needs to be grammatical, fluent, and retain the main ideas of its original counterpart without altering its meaning.
Do not modify the following words: {span}

Text: {text}
Object: {object}
Refers to: {span}

'''

In [111]:
request(samples, plain_english_instruction)

{'text': 'The manager asked the employee to not forget his car.', 'label': [['his', 'ambiguous']], 'object': 'his', 'span': 'ambiguous'}
{
  "type": "simplification",
  "modified_text": "The boss told the worker to remember the car.",
  "object": "his",
  "refer_to": "ambiguous",
  "rationale": "The word 'manager' was replaced with 'boss' for simplicity. 'Asked' was changed to 'told' to use a more common verb. 'Employee' was replaced with 'worker' as it is a simpler term. The phrase 'to not forget' was simplified to 'remember' for ease of understanding. The sentence was kept short and straightforward to avoid complexity, but the reference of 'his' remains ambiguous as it is unclear whether it refers to the boss or the worker."
}
{'text': 'After meeting with the producers, Sam went to their office.', 'label': [['their', 'ambiguous']], 'object': 'their', 'span': 'ambiguous'}
{
  "modifications": [
    {
      "type": "paraphrasing",
      "modified_text": "Sam had a meeting with the prod

KeyboardInterrupt: 

### Complex English

In [112]:
complex_english_instruction = '''Transform the original sentence into more complex English. 
Change some words with their more sophisticated technical synonym.
Change the sentence structure into more complex one.
Keep the modified sentence has nearly similar length with original one and do not alter its meaning.
Do not modify the following words: {span}

Text: {text}
Object: {object}
Refers to: {span}

'''

In [113]:
request(samples, complex_english_instruction)

{'text': 'The manager asked the employee to not forget his car.', 'label': [['his', 'ambiguous']], 'object': 'his', 'span': 'ambiguous'}
{
  "modifications": [
    {
      "type": "Complexity Increase",
      "modified_text": "The supervisor requested that the subordinate ensure the retention of the location of his automobile.",
      "object": "his",
      "refer_to": "ambiguous",
      "rationale": "The modification increases the complexity of the sentence by substituting 'manager' with 'supervisor', 'asked' with 'requested', 'employee' with 'subordinate', 'not forget' with 'ensure the retention of', and 'car' with 'automobile'. The sentence structure is made more complex by adding a subordinate clause and using a more formal tone."
    }
  ]
}
{'text': 'After meeting with the producers, Sam went to their office.', 'label': [['their', 'ambiguous']], 'object': 'their', 'span': 'ambiguous'}
{
  "modifications": [
    {
      "type": "Complexity Increase",
      "modified_text": "Subseq

KeyboardInterrupt: 

### Dialectal features

#### African American English

In [114]:
aave_english_instruction = '''Rewrite the text in African American Vernacular English. Do not modify the following words: {span}.

Text: {text}
Object: {object}
Refers to: {span}

'''

In [115]:
request(samples, aave_english_instruction)

{'text': 'The manager asked the employee to not forget his car.', 'label': [['his', 'ambiguous']], 'object': 'his', 'span': 'ambiguous'}
{
  "modifications": [
    {
      "type": "linguistic_variation",
      "modified_text": "The manager told the employee to remember his car.",
      "object": "his",
      "refer_to": "ambiguous",
      "rationale": "The sentence structure has been slightly altered to fit a more conversational tone, which may be more prevalent in African American Vernacular English (AAVE). The verb 'asked' is replaced with 'told' to reflect a more direct form of speech, and 'to not forget' is changed to 'to remember' for simplicity and flow. However, the referent of 'his' remains ambiguous as it could refer to either the manager or the employee."
    }
  ]
}
{'text': 'After meeting with the producers, Sam went to their office.', 'label': [['their', 'ambiguous']], 'object': 'their', 'span': 'ambiguous'}
{
  "type": "linguistic_variation_modification",
  "modified_text

KeyboardInterrupt: 

In [116]:
singlish_instruction = '''Rewrite the text in Singapore Colloquial English (Singlish, Basilect). Do not modify the following words: {span}.

Text: {text}
Object: {object}
Refers to: {span}

'''

In [117]:
request(samples, singlish_instruction)

{'text': 'The manager asked the employee to not forget his car.', 'label': [['his', 'ambiguous']], 'object': 'his', 'span': 'ambiguous'}
{
  "modification_1": {
    "type": "Dialectal Variation",
    "modified_text": "The manager ask the employee not to forget his car lah.",
    "object": "his",
    "refer_to": "ambiguous",
    "rationale": "The text has been modified to reflect characteristics of Singapore Colloquial English (Singlish), which often includes the use of 'lah' for emphasis and a more relaxed grammatical structure, such as 'ask' instead of 'asked'. The modification aims to maintain the ambiguity of the pronoun 'his' while altering the surrounding text to fit the Singlish dialect."
  }
}
{'text': 'After meeting with the producers, Sam went to their office.', 'label': [['their', 'ambiguous']], 'object': 'their', 'span': 'ambiguous'}
{
  "type": "linguistic_variation_modification",
  "modified_text": "After meeting with the producers, Sam went to their office lah.",
  "objec

KeyboardInterrupt: 

#### Indian English 

In [118]:
higlish_instruction = '''Rewrite the text using Indianisms and Indian English grammar. Do not modify the following words: {span}.

Text: {text}
Object: {object}
Refers to: {span}

'''

In [119]:
request(samples, higlish_instruction)

{'text': 'The manager asked the employee to not forget his car.', 'label': [['his', 'ambiguous']], 'object': 'his', 'span': 'ambiguous'}
{
  "modification_1": {
    "type": "Lexical and grammatical modification to Indian English",
    "modified_text": "The manager requested the employee to kindly remember his car.",
    "object": "his",
    "refer_to": "ambiguous",
    "rationale": "The phrase 'asked to not forget' has been changed to 'requested to kindly remember' which is a common polite request format in Indian English. The structure of the sentence has been slightly altered to fit the Indian English style while keeping the meaning intact."
  }
}
{'text': 'After meeting with the producers, Sam went to their office.', 'label': [['their', 'ambiguous']], 'object': 'their', 'span': 'ambiguous'}
{
  "modification_1": {
    "type": "Cultural and Linguistic Adaptation",
    "modified_text": "After having a meeting with the producers, Sam proceeded to their office only.",
    "object": "the

KeyboardInterrupt: 